# Multi-Agent Customer Service System with A2A and MCP

In [1]:
import os, signal, subprocess, sys

# Kill anything on 8001/8002/8003 (macOS)
for p in [8001, 8002, 8003]:
    out = subprocess.run(["lsof", "-ti", f"tcp:{p}"], capture_output=True, text=True).stdout.strip().split()
    for pid in out:
        try:
            os.kill(int(pid), signal.SIGKILL)
        except Exception:
            pass
print("Killed any old servers on 8001/8002/8003.")


Killed any old servers on 8001/8002/8003.


## Set API key

In [2]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")

# You can swap models via env var
os.environ.setdefault("OPENAI_MODEL", "gpt-4o-mini")

print("OPENAI_API_KEY set:", bool(os.environ.get("OPENAI_API_KEY")))
print("OPENAI_MODEL:", os.environ["OPENAI_MODEL"])


Enter OPENAI_API_KEY:  ········


OPENAI_API_KEY set: True
OPENAI_MODEL: gpt-4o-mini


In [3]:
import os, sys, pathlib

ROOT = pathlib.Path.cwd()
print("CWD:", ROOT)
assert (ROOT / "agents").exists()
assert (ROOT / "mcp_server.py").exists()
assert (ROOT / "database_setup.py").exists()

DATA_URL   = "http://127.0.0.1:8001"
SUPPORT_URL= "http://127.0.0.1:8002"
ROUTER_URL = "http://127.0.0.1:8003"

# Correct (non-deprecated) agent card endpoint
AGENT_CARD_PATH = "/.well-known/agent-card.json"

# JSON-RPC is POSTed to the server root
RPC_PATH = "/"

print("DATA_URL:", DATA_URL)
print("SUPPORT_URL:", SUPPORT_URL)
print("ROUTER_URL:", ROUTER_URL)
print("Agent card path:", AGENT_CARD_PATH)
print("RPC path:", RPC_PATH)


CWD: /Users/Devyani/msads/genai/hw5/multi-agent-customer-service
DATA_URL: http://127.0.0.1:8001
SUPPORT_URL: http://127.0.0.1:8002
ROUTER_URL: http://127.0.0.1:8003
Agent card path: /.well-known/agent-card.json
RPC path: /


In [4]:
import subprocess, sys

# Recreate support.db with test data
subprocess.run([sys.executable, "database_setup.py"], check=True)
print("Database reset complete.")


Connected to database: support.db
Tables created successfully!
Triggers created successfully!

DATABASE SCHEMA

CUSTOMERS TABLE:
------------------------------------------------------------
  id              INTEGER     
  name            TEXT       NOT NULL 
  email           TEXT        
  phone           TEXT        
  status          TEXT       NOT NULL DEFAULT 'active'
  created_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP
  updated_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP

TICKETS TABLE:
------------------------------------------------------------
  id              INTEGER     
  customer_id     INTEGER    NOT NULL 
  issue           TEXT       NOT NULL 
  status          TEXT       NOT NULL DEFAULT 'open'
  priority        TEXT       NOT NULL DEFAULT 'medium'
  created_at      DATETIME    DEFAULT CURRENT_TIMESTAMP

FOREIGN KEYS:
------------------------------------------------------------
  tickets.customer_id -> customers.id

Would you like to insert sample data? (y/n): E

In [5]:
import subprocess, sys, time, os, signal, pathlib

LOG_DIR = pathlib.Path("logs")
LOG_DIR.mkdir(exist_ok=True)

def start_uvicorn(module_app: str, port: int, name: str):
    out = open(LOG_DIR / f"{name}.out.log", "w")
    err = open(LOG_DIR / f"{name}.err.log", "w")
    cmd = [
        sys.executable, "-m", "uvicorn",
        module_app,
        "--host", "127.0.0.1",
        "--port", str(port),
        "--log-level", "info",
    ]
    p = subprocess.Popen(cmd, stdout=out, stderr=err, text=True)
    return p

# IMPORTANT: these must match what you validated:
DATA_APP   = "agents.data_agent_server:app"
SUPPORT_APP= "agents.support_agent_server:app"
ROUTER_APP = "agents.router_agent_server:app"

procs = {}
procs["data"]    = start_uvicorn(DATA_APP, 8001, "data")
procs["support"] = start_uvicorn(SUPPORT_APP, 8002, "support")
procs["router"]  = start_uvicorn(ROUTER_APP, 8003, "router")

print("Spawned:", {k: v.pid for k, v in procs.items()})
time.sleep(1.0)


Spawned: {'data': 96936, 'support': 96937, 'router': 96938}


In [6]:
import asyncio, httpx, time

async def wait_for_agent_card(base_url: str, timeout_s: float = 20.0):
    url = base_url + AGENT_CARD_PATH
    t0 = time.time()
    last = None
    async with httpx.AsyncClient(timeout=2.5) as client:
        while time.time() - t0 < timeout_s:
            try:
                r = await client.get(url)
                if r.status_code == 200:
                    return r.json()
                last = {"status": r.status_code, "text": r.text[:200]}
            except Exception as e:
                last = str(e)
            await asyncio.sleep(0.25)
    raise RuntimeError(f"Agent card not reachable: {url}. Last: {last}")

data_card   = await wait_for_agent_card(DATA_URL)
support_card= await wait_for_agent_card(SUPPORT_URL)
router_card = await wait_for_agent_card(ROUTER_URL)

print("DATA card name:", data_card.get("name"))
print("SUPPORT card name:", support_card.get("name"))
print("ROUTER card name:", router_card.get("name"))


DATA card name: Customer Data Agent
SUPPORT card name: Support Agent
ROUTER card name: Router Agent


In [7]:
import uuid, httpx, asyncio

def make_send_payload(text: str):
    return {
        "jsonrpc": "2.0",
        "id": str(uuid.uuid4()),
        "method": "message/send",
        "params": {
            "message": {
                "role": "user",
                "parts": [{"type": "text", "text": text}],
                "messageId": str(uuid.uuid4()),
            }
        },
    }

async def a2a_send(base_url: str, text: str):
    url = base_url + RPC_PATH
    payload = make_send_payload(text)
    async with httpx.AsyncClient(timeout=30.0) as client:
        r = await client.post(url, json=payload)
        r.raise_for_status()
        return r.json()

# Quick sanity ping to Router
router_ping = await a2a_send(ROUTER_URL, "hi")
router_ping


{'jsonrpc': '2.0',
 'id': 'bb32a053-de4e-4ff4-a994-b9b1c35ae003',
 'result': {'kind': 'message',
  'role': 'agent',
  'messageId': '24d406eb-7150-403f-bcb8-bdc91f3a2131',
  'parts': [{'kind': 'text',
    'text': "Routed to Support Agent:\nSupport Agent (triage) [2025-12-19T00:37:27Z]\nTell me what happened (refund, shipping, cancellation, damaged item, billing).\n\nA2A routing log:\n[router] time=2025-12-19T00:37:27Z\n[router] DATA_URL=http://127.0.0.1:8001/ SUPPORT_URL=http://127.0.0.1:8002/\n[router] SUPPORT call OK: 'hi'"}]}}

In [8]:
from pprint import pprint

tests = [
    # Scenario 1: Task allocation (router should call Data, then Support)
    "I need help with my account, customer ID 5",

    # Scenario 2: Negotiation / escalation (router should involve Support, and may request Data context)
    "I want to cancel my subscription but I'm having billing issues",

    # Scenario 3: Multi-step coordination (router decomposes work)
    "Show me all active customers who have open tickets",
]

results = []
for q in tests:
    print("\n" + "="*90)
    print("QUERY:", q)
    resp = await a2a_send(ROUTER_URL, q)
    pprint(resp)
    results.append((q, resp))



QUERY: I need help with my account, customer ID 5
{'id': 'be6b0982-f334-4c36-ac7e-95815ac5d8a6',
 'jsonrpc': '2.0',
 'result': {'kind': 'message',
            'messageId': 'aaf120dd-fe21-4c02-93aa-8ee56fbfa77f',
            'parts': [{'kind': 'text',
                       'text': 'Coordinated answer (Router -> Data then '
                               'Support):\n'
                               'Customer context from Data Agent:\n'
                               '{\n'
                               '  "found": true,\n'
                               '  "customer": {\n'
                               '    "id": 5,\n'
                               '    "name": "Charlie Brown",\n'
                               '    "email": "charlie.brown@email.com",\n'
                               '    "phone": "+1-555-0105",\n'
                               '    "status": "active",\n'
                               '    "created_at": "2025-12-18 07:12:45",\n'
                               '   

In [9]:
from pprint import pprint

q = "Get customer information for ID 3"
print("DIRECT DATA QUERY:", q)
resp = await a2a_send(DATA_URL, q)
pprint(resp)


DIRECT DATA QUERY: Get customer information for ID 3
{'id': '81978cce-0334-4912-be9d-7f8f7c1c40eb',
 'jsonrpc': '2.0',
 'result': {'kind': 'message',
            'messageId': '97fe2883-f7c5-44a3-a353-7185bf8a8c53',
            'parts': [{'kind': 'text',
                       'text': '{\n'
                               '  "found": true,\n'
                               '  "customer": {\n'
                               '    "id": 3,\n'
                               '    "name": "Bob Johnson",\n'
                               '    "email": "bob.johnson@example.com",\n'
                               '    "phone": "+1-555-0103",\n'
                               '    "status": "disabled",\n'
                               '    "created_at": "2025-12-18 07:12:45",\n'
                               '    "updated_at": "2025-12-18 07:12:45"\n'
                               '  }\n'
                               '}'}],
            'role': 'agent'}}


## Cleanup: stop the servers

In [10]:
import os, signal, time

for name, p in procs.items():
    if p.poll() is None:
        print("Stopping", name, "pid", p.pid)
        p.send_signal(signal.SIGINT)

time.sleep(1.0)

for name, p in procs.items():
    if p.poll() is None:
        print("Force-killing", name, "pid", p.pid)
        p.kill()

print("Done. Logs are in ./logs/")


Stopping data pid 96936
Stopping support pid 96937
Stopping router pid 96938
Done. Logs are in ./logs/
